In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook, tnrange
import gc
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import roc_auc_score
from scipy.sparse import hstack,vstack
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
tic = time.time()
invite_info = pd.read_table('../data/data_set_0926/invite_info_0926.txt',header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']
    
invite_info_evaluate = pd.read_table('../data/data_set_0926/invite_info_evaluate_2_0926.txt',header=None)
invite_info_evaluate.columns =  ['问题ID','用户ID','邀请创建时间']
    
member_info = pd.read_table('../data/data_set_0926/member_info_0926.txt',header=None)
member_info.columns = ['用户ID','性别','创作关键词的编码序列','创作数量等级','创作热度等级','注册类型','注册平台','访问频率','用户二分类特征A','用户二分类特征B','用户二分类特征C','用户二分类特征D','用户二分类特征E','用户分类特征A','用户分类特征B','用户分类特征C','用户分类特征D','用户分类特征E','用户的盐值分数','用户关注的话题','用户感兴趣的话题']
data = pd.concat([invite_info, invite_info_evaluate], axis=0).reset_index(drop=True)
data = data.merge(member_info, on='用户ID', how='left')
del member_info,invite_info_evaluate,invite_info
gc.collect()

print("Used time: %d s" % (time.time()-tic))

Used time: 66 s


In [3]:
lbl_enc_feat = []
for feat in tqdm(['性别','访问频率','用户分类特征A', '用户分类特征B', '用户分类特征C', '用户分类特征D','用户分类特征E']):
    data['{}_lbl_enc'.format(feat)] = LabelEncoder().fit_transform(data[feat].astype(str))
    lbl_enc_feat.append('{}_lbl_enc'.format(feat))
        
useless_feat = ['创作关键词的编码序列', '创作数量等级','创作热度等级', '注册类型', '注册平台']
data.drop(useless_feat, axis=1, inplace=True)

100%|██████████| 7/7 [01:42<00:00, 14.69s/it]


In [4]:
tmp = pd.read_pickle('./features/ques_topicID_svd_30_new.pickle')#
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp
    
tmp = pd.read_pickle('./features/user_fav_topic_svd_30_new.pickle')#
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp
    
tmp = pd.read_pickle('./features/user_watched_topic_svd_30_new.pickle')#
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp

In [5]:
tmp = pd.read_pickle('./features/inv_ans_gap.pickle')
tmp = tmp.reset_index(drop=True)
data = data.reset_index(drop=True)
data[list(tmp.columns)] = tmp

In [6]:
tmp = pd.read_pickle('./features/prev_ans_ques_title_sim.pickle')
tmp = tmp.reset_index(drop=True)
data = data.reset_index(drop=True)
data[list(tmp.columns)] = tmp

In [7]:
tmp = pd.read_pickle('./features/strong_feat_collections.pickle')#过采样#
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp

In [8]:
tmp = pd.read_pickle('./features/last_next_expo_feat.pickle')#过采样
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp

In [9]:
tmp = pd.read_pickle('./features/len_feat.pickle')#
data[list(tmp.columns)] = tmp

In [10]:
tmp = pd.read_pickle('./features/user_quesIDs_svd_new.pickle')#
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp

tmp = pd.read_pickle('./features/ques_userIDs_svd_new.pickle')#
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp

In [13]:
tmp = pd.read_pickle('./features/invite_time_stats.pickle')#
data[list(tmp.columns)] = tmp

In [14]:
tmp = pd.read_pickle('./features/user_last_answers.pickle')#过采样
tmp = tmp.iloc[0:data.shape[0]]
data[list(tmp.columns)] = tmp

In [15]:
tmp = pd.read_pickle('./features/user_ques_count.pickle')#
data[list(tmp.columns)] = tmp

In [16]:
tmp = pd.read_pickle('./features/same_topic_id_feat.pickle')#
data[list(tmp.columns)] = tmp

In [17]:
tmp = np.load('./features/ques_user_fav_topic_sim.npy')
data['问题_用户感兴趣_topic_sim'] = tmp
tmp = np.load('./features/ques_user_watch_topic_sim.npy')
data['问题_用户关注_topic_sim'] = tmp
tmp = np.load('./features/user_watch_fav_topic_sim.npy')
data['用户关注_感兴趣_topic_sim'] = tmp
tmp = np.load('./features/ques_topic_answer_SW_sim.npy')
data['问题标题_曾经回答_SW_sim'] = tmp
tmp = np.load('./features/ques_describe_answer_SW_sim.npy')
data['问题描述_曾经回答_SW_sim'] = tmp
tmp = np.load('./features/ques_topic_answer_W_sim.npy')
data['问题标题_曾经回答_W_sim'] = tmp
tmp = np.load('./features/ques_describe_answer_W_sim.npy')
data['问题描述_曾经回答_W_sim'] = tmp
tmp = np.load('./features/prev_ans_ques_topic_sim.npy')
data['问题描述_曾经回答_Topic_sim'] = tmp

In [18]:
tmp = pd.read_pickle('./features/answer_last_stat.pkl')
data[list(tmp.columns)] = tmp
tmp = pd.read_pickle('./features/answer_last_stat_I.pkl')
data[list(tmp.columns)] = tmp

In [19]:
tmp = pd.read_pickle('./features/pres_ans_time_stats.pkl')
data[list(tmp.columns)] = tmp

In [20]:
#scp -P 45170 lixiaoxiao@frp.mcurobot.com:workspace/comp_collection/Zhihu/tiny/features/lstm_enc_feat.pickle ./code/Zhihu/travis/features
tmp = pd.read_pickle('./features/lstm_enc_feat.pickle')#
data[list(tmp.columns)] = tmp

#scp -P 45170 lixiaoxiao@frp.mcurobot.com:workspace/comp_collection/Zhihu/tiny/features/w2v_sum_nn.pkl ./code/Zhihu/travis/features
tmp = pd.read_pickle('./features/w2v_sum_nn.pkl')##
data[list(tmp.columns)] = tmp

In [ ]:
#cchan feat
#scp -P 51456 root@111.44.254.173:/root/code/Zhihu2/cchan/feats/inv_times_st.pkl ./code/Zhihu/travis/features
#XRSEvxjm
tmp = pd.read_pickle('./features/prev_topic_sims_stat.pkl')
data[list(tmp.columns)] = tmp

tmp = pd.read_pickle('./features/local_user_ques_expo.pkl')
feat_I =['question_curr_expo','user_curr_expo']
data[feat_I] = tmp[feat_I]

tmp = pd.read_pickle('./features/inv_times_st.pkl')
feat_II = ['prev_ans_times_min_gap', 'prev_ans_times_mean_gap', 'qtime_std','qtime_mean', 'utime_std', 'utime_mean', 'iweek', 'qlast_itime_gap','qllast_itime_gap', 'qlllast_itime_gap', 'qnext_itime_gap','qnnext_itime_gap', 'qnnnext_itime_gap', 'ulast_itime_gap','ullast_itime_gap', 'ulllast_itime_gap', 'unext_itime_gap','unnext_itime_gap', 'unnnext_itime_gap']
data[feat_II] = tmp[feat_II]

In [ ]:
tmp = pd.read_pickle('./features/ques_len_stat_feat.pkl')##
new_test_feat = list(tmp.columns)
data[list(tmp.columns)] = tmp

In [ ]:
data.to_pickle('./data/data_all.pickle')